In [1]:
# default_exp eval

# Eval

> Model Evaluation

In [2]:
#hide
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import *
import warnings
warnings.filterwarnings("ignore")

In [3]:
# export
import os
import torch
from torch.utils.data import DataLoader
from ti.dataloader import DatasetTraj, zero_padding, getSTW, splitData, file_dir
from ti.prep import Transformer
from ti.model import Difference, TrajectoryDN
import torch.nn as nn
from ti.train import test

BATCH_SIZE = 4
threshold = 0.5
params = {
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'collate_fn': zero_padding
}
mode = 'real'
stw = getSTW(mode=mode, is_train=False)
ts_range = range(len(stw))
# Partitions
mode = 'real'
# Generators
test_set = DatasetTraj(ts_range, stw, mode=mode)
test_g = DataLoader(test_set, **params)
transformer = Transformer()
diff_net = Difference(mode='simple')
net = TrajectoryDN(diff_net, n_features=(len(transformer.features_traj))*2) # 2x for org and dest 
net = nn.DataParallel(net)
criterion = nn.BCELoss()
print(net)

DataParallel(
  (module): TrajectoryDN(
    (diff_net): Difference()
    (lstm): LSTM(8, 64, num_layers=2, batch_first=True)
    (fcn): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
      (4): Linear(in_features=32, out_features=1, bias=True)
      (5): Sigmoid()
    )
  )
)


In [4]:
# export
MODEL_SAVE_PATH = os.path.join(file_dir, '../runs', 'run_tran_normalsubtraj_Batch4_EPOCHS1000', f'Epoch{999}.pth')
print('Loading pre-trained model and optimizer weights from ', MODEL_SAVE_PATH)
checkpoint = torch.load(MODEL_SAVE_PATH)
# print(checkpoint['model_state_dict'])
net.load_state_dict(checkpoint['model_state_dict'])
# optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
lengths = [len(i) for i in stw]
avg = sum(lengths)/len(stw)
print(f'Iterations: {avg}')
loss_list, acc_list, tp_list, tr_list, tf_list = list(), list(), list(), list(), list()
for i in range(int(10)):
    test_loss, test_acc, tp, tr, tf = test(net, test_g, criterion, threshold, mtype='DN')
    loss_list.append(test_loss)
    acc_list.append(test_acc)
    tp_list.append(tp)
    tr_list.append(tr)
    tf_list.append(tf)
    
from statistics import mean
# print(test_loss, test_acc, tp, tr, tf)
print(mean(loss_list), mean(acc_list), mean(tp_list), mean(tr_list), mean(tf_list))

Loading pre-trained model and optimizer weights from  /Users/mahantesh/data/2022/DS/A3/git_workspace/DifferenceNet/ti/ti/../runs/run_tran_normalsubtraj_Batch4_EPOCHS1000/Epoch999.pth
Iterations: 2.2128801431127014
0.48159 0.75 0.762 0.75 0.749


# Export -

In [5]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_prep.ipynb.
Converted 01_dataloader.ipynb.
Converted 02_model.ipynb.
Converted 03_train.ipynb.
Converted 04_eval.ipynb.
Converted index.ipynb.
